# Energy Mix & Carbon Emission Estimates

In [1]:
from IPython.display import clear_output

In [2]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests

clear_output()

In [3]:
google_drive_path = "/content/drive/MyDrive/LINC/"

In [4]:
# Import libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [5]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

## 1.Energy Mix

### 1.1.Cyprus

In [6]:
from datetime import date

#### 1.1.1. Current date

In [7]:
def get_cy_temp_forecast(lati, longi, suffix, past_days = 0, forecast_days = 1):

  url = "https://api.open-meteo.com/v1/forecast"
  params = {
	  "latitude": lati,
	  "longitude": longi,
    "hourly": "direct_normal_irradiance",
    "timezone": "Europe/Moscow",
    "past_days": past_days,
    "forecast_days": forecast_days
  }

  responses = openmeteo.weather_api(url, params=params)

  # Process first location. Add a for-loop for multiple locations or weather models
  response = responses[0]

  # Process hourly data. The order of variables needs to be the same as requested.
  hourly = response.Hourly()
  hourly_direct_normal_irradiance = hourly.Variables(0).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
	  start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	  end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	  freq = pd.Timedelta(seconds = hourly.Interval()),
	  inclusive = "left"
  )}

  hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance

  hourly_dataframe = pd.DataFrame(data = hourly_data)
  hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
  hourly_dataframe['date'] = hourly_dataframe['date'].dt.tz_convert('Europe/Athens')


  # Adding suffix to columns 2 to 4
  for col in hourly_dataframe.columns[1:]:  # Columns 2 to end
    hourly_dataframe.rename(columns={col: col + suffix}, inplace=True)

  return hourly_dataframe


In [8]:
def get_cy_current_energy_mix_data(num_past_days = 3):

  # Define the start and end dates
  start_date = date.today()

  df = pd.DataFrame()

  url = "https://tsoc.org.cy/electrical-system/archive-total-daily-system-generation-on-the-transmission-system/?startdt=" + \
        start_date.strftime("%d-%m-%Y") + "&enddt=%2B1days"

  # Fetch the data
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
    # Assuming the data is in a table format in the HTML
    df = pd.read_html(response.text)[1]
  else:
      print(f"Failed to fetch data. Status code: {response.status_code}")

  df.columns = ['timestamp', 'total_conv_avail', 'wind', 'renewable', "total", "conventional"]

  df = df[['timestamp', "total", "wind", "renewable", "conventional"]]

  df['timestamp'] =  pd.to_datetime(df['timestamp']).dt.tz_localize('Europe/Athens', ambiguous = "NaT", nonexistent = "NaT")

  # Merge with hourly solar irradiance values
  df = pd.merge(
      df,
      get_cy_temp_forecast(35.19, 33.38, "_nic"),
      left_on = 'timestamp', right_on = "date", how = "left"
  )

  # Assumption all conventional productions comes from oil
  df['oil'] = df['conventional']
  df['gas'] = 0
  df['coal'] = 0

  # Assumption: no sunshine at midnight => all renewable comes from biomass => this value is assumed throughout the day
  biomass_value = df['renewable'].iloc[0]
  df['biomass'] = df.apply(lambda row: min(row.renewable, biomass_value), axis=1)

  df['pv'] = df.apply(lambda row: row.renewable - row.biomass if row.direct_normal_irradiance_nic >0 else 0, axis = 1)

  df['water'] = 0

  df = df.dropna(axis=0, how='any')

  return df[['timestamp', 'total', 'wind', 'oil', 'gas', 'coal', 'biomass', 'pv', 'water']].tail(1)

In [9]:
get_cy_current_energy_mix_data()

,timestamp,total,wind,oil,gas,coal,biomass,pv,water
40,2024-07-13 10:00:00+03:00,899.0,7.0,478.0,0,0,10.0,403.0,0


#### 1.1.2. Past 3 days

In [10]:
def get_cy_past3days_energy_mix_data(num_past_days = 3):

  # Define the start and end dates
  end_date = date.today()
  start_date = end_date - timedelta(days=3)

  # Calculate the difference in days
  delta = end_date - start_date

  df = pd.DataFrame()

  # Iterate over the range of days
  for i in range(delta.days + 1):

    day = start_date + timedelta(days=i)
    url = "https://tsoc.org.cy/electrical-system/archive-total-daily-system-generation-on-the-transmission-system/?startdt=" + day.strftime("%d-%m-%Y") + "&enddt=%2B1days"

    # Fetch the data
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
      # Assuming the data is in a table format in the HTML
      df = pd.concat([df, pd.read_html(response.text)[1]], ignore_index=True)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

  df.columns = ['timestamp', 'total_conv_avail', 'wind', 'renewable', "total", "conventional"]

  df = df[['timestamp', "total", "wind", "renewable", "conventional"]]

  df['timestamp'] =  pd.to_datetime(df['timestamp']).dt.tz_localize('Europe/Athens', ambiguous = "NaT", nonexistent = "NaT")

  # Merge with hourly solar irradiance values
  df = pd.merge(
      df,
      get_cy_temp_forecast(35.19, 33.38, "_nic", 3, 1),
      left_on = 'timestamp', right_on = "date", how = "right"
  )

  # Drop rows with missing values (intention: delete rows for the current day for hours still to come)
  df = df.dropna(axis=0, how='any')

  # Assumption all conventional productions comes from oil
  df['oil'] = df['conventional']
  df['gas'] = 0
  df['coal'] = 0

  # Assumption: no sunshine at midnight => all renewable comes from biomass => this value is assumed throughout the day
  biomass_value = df['renewable'].iloc[0]
  df['biomass'] = df.apply(lambda row: min(row.renewable, biomass_value), axis=1)

  df['pv'] = df.apply(lambda row: row.renewable - row.biomass if row.direct_normal_irradiance_nic >0 else 0, axis = 1)

  df['water'] = 0

  return df[['timestamp', 'total', 'wind', 'oil', 'gas', 'coal', 'biomass', 'pv', 'water']]

In [11]:
get_cy_past3days_energy_mix_data()

,timestamp,total,wind,oil,gas,coal,biomass,pv,water
0,2024-07-10 00:00:00+03:00,899.0,10.0,879.0,0,0,10.0,0.0,0
1,2024-07-10 01:00:00+03:00,832.0,8.0,813.0,0,0,10.0,0.0,0
2,2024-07-10 02:00:00+03:00,780.0,0.0,770.0,0,0,10.0,0.0,0
3,2024-07-10 03:00:00+03:00,749.0,0.0,739.0,0,0,10.0,0.0,0
4,2024-07-10 04:00:00+03:00,720.0,2.0,708.0,0,0,10.0,0.0,0
...,...,...,...,...,...,...,...,...,...
78,2024-07-13 06:00:00+03:00,652.0,38.0,603.0,0,0,10.0,0.0,0
79,2024-07-13 07:00:00+03:00,689.0,24.0,603.0,0,0,10.0,52.0,0
80,2024-07-13 08:00:00+03:00,777.0,11.0,577.0,0,0,10.0,179.0,0
81,2024-07-13 09:00:00+03:00,853.0,7.0,529.0,0,0,10.0,307.0,0


#### 1.1.3. Historical data

In [12]:
def get_cy_temp_historical(lati, longi, suffix, start_date, end_date):

  url = "https://archive-api.open-meteo.com/v1/archive"
  params = {
	  "latitude": lati,
	  "longitude": longi,
	  "start_date": start_date,
	  "end_date": end_date,
    "hourly": "direct_normal_irradiance",
	  "timezone": "Europe/Moscow"
  }

  responses = openmeteo.weather_api(url, params=params)

  # Process first location. Add a for-loop for multiple locations or weather models
  response = responses[0]

  # Process hourly data. The order of variables needs to be the same as requested.
  hourly = response.Hourly()
  hourly_direct_normal_irradiance = hourly.Variables(0).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
	  start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	  end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	  freq = pd.Timedelta(seconds = hourly.Interval()),
	  inclusive = "left"
  )}

  hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance

  hourly_dataframe = pd.DataFrame(data = hourly_data)
  hourly_dataframe['date'] = hourly_dataframe['date'].dt.tz_convert('Europe/Athens')

  # Adding suffix to columns 2 to 4
  for col in hourly_dataframe.columns[1:]:  # Columns 2 to end
    hourly_dataframe.rename(columns={col: col + suffix}, inplace=True)

  return hourly_dataframe


In [13]:
def get_cy_hist_energy_mix_data(start_yyyy, start_mm, start_dd, end_yyyy, end_mm, end_dd):

  # Define the start and end dates
  start_date = datetime(int(start_yyyy), int(start_mm), int(start_dd))
  end_date = datetime(int(end_yyyy), int(end_mm), int(end_dd))
  # print(start_date.strftime("%Y-%m-%d"))

  # Calculate the difference in days
  delta = end_date - start_date

  df = pd.DataFrame()

  # Iterate over the range of days
  for i in range(delta.days + 1):

    day = start_date + timedelta(days=i)
    url = "https://tsoc.org.cy/electrical-system/archive-total-daily-system-generation-on-the-transmission-system/?startdt=" + day.strftime("%d-%m-%Y") + "&enddt=%2B1days"

    # Fetch the data
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
      # Assuming the data is in a table format in the HTML
      df = pd.concat([df, pd.read_html(response.text)[1]], ignore_index=True)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

  df.columns = ['timestamp', 'total_conv_avail', 'wind', 'renewable', "total", "conventional"]

  df = df[['timestamp', "total", "wind", "renewable", "conventional"]]

  df['timestamp'] =  pd.to_datetime(df['timestamp']).dt.tz_localize('Europe/Athens', ambiguous = "NaT", nonexistent = "NaT")

  # Merge with hourly solar irradiance values
  df = pd.merge(
      df,
      get_cy_temp_historical(35.19, 33.38, "_nic", start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")),
      left_on = 'timestamp', right_on = "date", how = "right"
  )

  # Drop rows with missing values (intention: delete rows for the current day for hours still to come)
  df = df.dropna(axis=0, how='any')

  # Assumption all conventional productions comes from oil
  df['oil'] = df['conventional']
  df['gas'] = 0
  df['coal'] = 0

  # Assumption: no sunshine at midnight => all renewable comes from biomass => this value is assumed throughout the day
  biomass_value = df['renewable'].iloc[0]
  df['biomass'] = df.apply(lambda row: min(row.renewable, biomass_value), axis=1)

  df['pv'] = df.apply(lambda row: row.renewable - row.biomass if row.direct_normal_irradiance_nic >0 else 0, axis = 1)

  df['water'] = 0

  return df[['timestamp', 'total', 'wind', 'oil', 'gas', 'coal', 'biomass', 'pv', 'water']]

In [157]:
# 01/01/2022-30/05/2024
df_cy_historical = get_cy_hist_energy_mix_data(2022, 1, 1, 2024, 5, 30)
df_cy_historical.to_csv(google_drive_path + "EnergyMix_CY_historical_estimated_01-01-2022_30-05-2024.csv", index = False)

In [158]:
df_cy_historical

,timestamp,total,wind,oil,gas,coal,biomass,pv,water
1,2022-01-01 00:00:00+02:00,556.0,78.0,471.0,0,0,6.0,0.0,0
2,2022-01-01 01:00:00+02:00,517.0,70.0,441.0,0,0,6.0,0.0,0
3,2022-01-01 02:00:00+02:00,479.0,57.0,416.0,0,0,6.0,0.0,0
4,2022-01-01 03:00:00+02:00,433.0,40.0,387.0,0,0,6.0,0.0,0
5,2022-01-01 04:00:00+02:00,404.0,45.0,353.0,0,0,6.0,0.0,0
...,...,...,...,...,...,...,...,...,...
21139,2024-05-30 19:00:00+03:00,621.0,48.0,530.0,0,0,6.0,36.0,0
21140,2024-05-30 20:00:00+03:00,618.0,60.0,549.0,0,0,6.0,3.0,0
21141,2024-05-30 21:00:00+03:00,620.0,53.0,558.0,0,0,6.0,0.0,0
21142,2024-05-30 22:00:00+03:00,573.0,52.0,512.0,0,0,6.0,0.0,0


### 1.2.Poland

#### 1.2.1.Live data

In [ ]:
import requests
import time

In [ ]:
def fetch_json(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response code was unsuccessful
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [ ]:
def get_pol_live_energy_mix_data():

  url = "https://www.pse.pl/transmissionMapService"

  json_file = fetch_json(url)

  if json_file:

    df = pd.DataFrame([(
        json_file['timestamp'],
        json_file['data']['podsumowanie']['generacja'], # total_generation
        json_file['data']['podsumowanie']['wiatrowe'], # wind power

        json_file['data']['podsumowanie']['cieplne'], #thermal_power

        json_file['data']['podsumowanie']['PV'], # pv power

        json_file['data']['podsumowanie']['wodne'], # water power

        # Assumption: Other renewables -> biomass
        json_file['data']['podsumowanie']['inne'], #other renewables (assummed to be biomass)

        # json_file['data']['podsumowanie']['zapotrzebowanie'], #total_demand
        # json_file['data']['podsumowanie']['czestotliwosc'] # frequency
    )])

  df.columns = ['timestamp', "total", "wind", "thermal", "pv", "water", "biomass"]

  # https://notesfrompoland.com/2024/01/03/poland-produced-record-26-of-electricity-from-renewables-in-2023/
  # However, coal continued to account for almost two thirds of power production in Poland, with hard coal generating
  # 65.27 TWh (43%) and brown coal (lignite) 31.47 TWh (20.8%). Gas accounted for a further 12.94 TWh (8.5%)
  # and oil 2.54 TWh (1.7%).

  df['oil'] = round((2.54) / (65.27 + 31.47 + 12.94 + 2.54) * df['thermal'], 0)
  df['oil'] = df['oil'].apply(lambda x: int(x))
  df['gas'] = round((12.94) / (65.27 + 31.47 + 12.94 + 2.54) * df['thermal'], 0)
  df['gas'] = df['gas'].apply(lambda x: int(x))
  df['coal'] = round((65.27 + 31.47) / (65.27 + 31.47 + 12.94 + 2.54) * df['thermal'], 0)
  df['coal'] = df['coal'].apply(lambda x: int(x))

  # df['timestamp'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))

  return df[['timestamp', 'total', 'wind', 'oil', 'gas', 'coal', 'biomass', 'pv', 'water']]

In [ ]:
get_pol_live_energy_mix_data()

#### 1.2.2.Historical data

In [ ]:
# # Source: https://www.pse.pl/web/pse-eng/data/polish-power-system-operation/generation-in-wind-farms
# df_renewable = pd.concat(
#     [
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_G_WF_20231103_20231130_20231201003525.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_G_WF_20231201_20231231_20240101003525.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_G_WF_20240101_20240131_20240201003516.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_G_WF_20240201_20240209_20240210003524.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_G_WF_20240418_20240504_20240505003524.csv", sep = ";")
#     ],
#     axis=0, ignore_index=True)

# df_renewable.shape

In [ ]:
# # Source: https://www.pse.pl/web/pse-eng/data/polish-power-system-operation/basic-data

# df_total = pd.concat(
#     [
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_BD_20231103_20231130_20231201010847.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_BD_20231201_20231231_20240101010530.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_BD_20240101_20240131_20240201010528.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_BD_20240201_20240209_20240210010518.csv", sep = ";"),
#         pd.read_csv(google_drive_path + "PL_EnergyMix_historical_data_raw/EN_Operation_PPS_BD_20240418_20240504_20240505010541.csv", sep = ";")
#     ],
#     axis=0, ignore_index=True)

# df_total.shape

In [ ]:
import glob

In [ ]:
# Source: https://www.pse.pl/web/pse-eng/data/polish-power-system-operation/basic-data

# Specify the directory path
directory_path = google_drive_path + "PL_EnergyMix_historical_data_raw/"

# Find all CSV files that start with 'EN_Operation_PPS_BD'
csv_files = glob.glob(f"{directory_path}/EN_Operation_PPS_BD*.csv")

# Read and concatenate all CSV files
dataframes = [pd.read_csv(file, sep = ';') for file in csv_files]

df_total = pd.concat(dataframes, ignore_index=True)
df_total

,Date,Time,Polish power demand,JWCD total generation,nJWCD total generation,Total cross-border electricity exchange with synchronous zone,Total cross-border exchange with non synchronous zone
0,2023-01-01,1,13271.825,5197.425,9247.013,-1084.450,-72.575
1,2023-01-01,2,12508.713,5746.300,7944.500,-1229.713,65.238
2,2023-01-01,3,11845.200,6632.363,6808.400,-1542.850,94.163
3,2023-01-01,4,11462.400,6273.838,7154.650,-1923.838,294.900
4,2023-01-01,5,11454.200,5713.238,7929.300,-1996.100,178.438
...,...,...,...,...,...,...,...
12402,2024-05-31,20,18818.750,12064.475,4543.050,2081.388,129.588
12403,2024-05-31,21,18757.688,12492.275,4181.438,1962.925,124.113
12404,2024-05-31,22,18271.750,12077.238,4149.675,1939.100,123.200
12405,2024-05-31,23,17183.263,10592.625,4337.538,2062.650,204.750


In [ ]:
# Source: https://www.pse.pl/web/pse-eng/data/polish-power-system-operation/generation-in-wind-farms

# Specify the directory path
directory_path = google_drive_path + "PL_EnergyMix_historical_data_raw/"

# Find all CSV files that start with 'EN_Operation_PPS_G'
csv_files = glob.glob(f"{directory_path}/EN_Operation_PPS_G*.csv")

# Read and concatenate all CSV files
dataframes = [pd.read_csv(file, sep = ';') for file in csv_files]

df_renewable = pd.concat(dataframes, ignore_index=True)
df_renewable

,Date,Time,Generation of Wind Farms,Generation of Solar Farms
0,2023-01-01,1,5529.188,0.000
1,2023-01-01,2,4250.700,0.000
2,2023-01-01,3,3124.613,0.000
3,2023-01-01,4,3486.663,0.000
4,2023-01-01,5,4257.800,0.000
...,...,...,...,...
12402,2024-05-31,20,770.913,927.988
12403,2024-05-31,21,966.775,384.225
12404,2024-05-31,22,1327.963,13.450
12405,2024-05-31,23,1568.638,0.000


In [ ]:
import pandas as pd
import pytz

# Define timezone
timezone = 'Europe/Warsaw'  # Replace with your timezone

# Define start and end timestamps
start_timestamp = pd.Timestamp('2023-01-01 01:00:00', tz=timezone)
end_timestamp = pd.Timestamp('2024-06-01 00:00:00', tz=timezone)

# Generate hourly datetime range
hourly_range = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')
hourly_range

DatetimeIndex(['2023-01-01 01:00:00+01:00', '2023-01-01 02:00:00+01:00',
               '2023-01-01 03:00:00+01:00', '2023-01-01 04:00:00+01:00',
               '2023-01-01 05:00:00+01:00', '2023-01-01 06:00:00+01:00',
               '2023-01-01 07:00:00+01:00', '2023-01-01 08:00:00+01:00',
               '2023-01-01 09:00:00+01:00', '2023-01-01 10:00:00+01:00',
               ...
               '2024-05-31 15:00:00+02:00', '2024-05-31 16:00:00+02:00',
               '2024-05-31 17:00:00+02:00', '2024-05-31 18:00:00+02:00',
               '2024-05-31 19:00:00+02:00', '2024-05-31 20:00:00+02:00',
               '2024-05-31 21:00:00+02:00', '2024-05-31 22:00:00+02:00',
               '2024-05-31 23:00:00+02:00', '2024-06-01 00:00:00+02:00'],
              dtype='datetime64[ns, Europe/Warsaw]', length=12407, freq='H')

In [ ]:
len(hourly_range)

12407

In [ ]:
df_PL_hist = pd.merge(df_total, df_renewable, on= ['Date', 'Time'], how='inner')
df_PL_hist ['DateTime'] = hourly_range
df_PL_hist = df_PL_hist.drop(columns = ['Date', 'Time'])

df_PL_hist

,Polish power demand,JWCD total generation,nJWCD total generation,Total cross-border electricity exchange with synchronous zone,Total cross-border exchange with non synchronous zone,Generation of Wind Farms,Generation of Solar Farms,DateTime
0,13271.825,5197.425,9247.013,-1084.450,-72.575,5529.188,0.000,2023-01-01 01:00:00+01:00
1,12508.713,5746.300,7944.500,-1229.713,65.238,4250.700,0.000,2023-01-01 02:00:00+01:00
2,11845.200,6632.363,6808.400,-1542.850,94.163,3124.613,0.000,2023-01-01 03:00:00+01:00
3,11462.400,6273.838,7154.650,-1923.838,294.900,3486.663,0.000,2023-01-01 04:00:00+01:00
4,11454.200,5713.238,7929.300,-1996.100,178.438,4257.800,0.000,2023-01-01 05:00:00+01:00
...,...,...,...,...,...,...,...,...
12402,18818.750,12064.475,4543.050,2081.388,129.588,770.913,927.988,2024-05-31 20:00:00+02:00
12403,18757.688,12492.275,4181.438,1962.925,124.113,966.775,384.225,2024-05-31 21:00:00+02:00
12404,18271.750,12077.238,4149.675,1939.100,123.200,1327.963,13.450,2024-05-31 22:00:00+02:00
12405,17183.263,10592.625,4337.538,2062.650,204.750,1568.638,0.000,2024-05-31 23:00:00+02:00


In [ ]:
df_PL_hist_final = df_PL_hist.copy()
df_PL_hist_final['total_production'] = df_PL_hist_final['JWCD total generation'] + df_PL_hist_final['nJWCD total generation']
df_PL_hist_final = df_PL_hist_final[['total_production', 'Generation of Wind Farms', 'Generation of Solar Farms', 'DateTime']]
df_PL_hist_final.columns = ['total_production', 'wind', 'solar', 'timestamp']
df_PL_hist_final.to_csv(google_drive_path + "EnergyMix_PL_historical_total_wind_solar.csv", index = False)
df_PL_hist_final

,total_production,wind,solar,timestamp
0,14444.438,5529.188,0.000,2023-01-01 01:00:00+01:00
1,13690.800,4250.700,0.000,2023-01-01 02:00:00+01:00
2,13440.763,3124.613,0.000,2023-01-01 03:00:00+01:00
3,13428.488,3486.663,0.000,2023-01-01 04:00:00+01:00
4,13642.538,4257.800,0.000,2023-01-01 05:00:00+01:00
...,...,...,...,...
12402,16607.525,770.913,927.988,2024-05-31 20:00:00+02:00
12403,16673.713,966.775,384.225,2024-05-31 21:00:00+02:00
12404,16226.913,1327.963,13.450,2024-05-31 22:00:00+02:00
12405,14930.163,1568.638,0.000,2024-05-31 23:00:00+02:00


In [ ]:
dpd.merge(df_total, df_renewable, on= ['Date', 'Time'], how='inner').\
to_csv(google_drive_path + "EnergyMix_PL_historical_raw.csv", index = False)

In [ ]:
df_PL_hist = pd.read_csv(google_drive_path + "EnergyMix_PL_historical_raw.csv")

# ------------------------------------------------------------------------------------------------------------
# CONVERT DATE & TIME COLUMNS INTO A SINGLE TIMESTAMP COLUMN
# ------------------------------------------------------------------------------------------------------------

# Ensure Date is in datetime format
df_PL_hist['Date'] = pd.to_datetime(df_PL_hist['Date'])

# Function to adjust time and date
def adjust_time(row):
    if row['Time'] == 24:
        row['Time'] = '00:00:00'
        row['Date'] += pd.Timedelta(days=1)
    else:
        row['Time'] = f"{row['Time']:02}:00:00"
    return row

# Apply the adjustment function
df_PL_hist = df_PL_hist.apply(adjust_time, axis=1)

# Combine Date and Time into a single datetime column
df_PL_hist['timestamp'] = pd.to_datetime(df_PL_hist['Date'].dt.strftime('%Y-%m-%d') + ' ' + df_PL_hist['Time'])


# ------------------------------------------------------------------------------------------------------------
# GET THE ENERGY MIX
# ------------------------------------------------------------------------------------------------------------

# Assuptions:
# 1) For simplicity purposes, the carbon footprint of the electricity imported through interconnections will be neglected.
# 2) For renewable sources of electrcity, there were historical data only for wind and solar farm energy generation.
#    Hydro power was assumed to be a constant of 1.3% of the total electrcity generation based on the following article:
#    https://notesfrompoland.com/2024/01/03/poland-produced-record-26-of-electricity-from-renewables-in-2023/
#    The rest of the electricity generation comes from conventional power sources (oil, gas, coal)
# 3) The themal power will be attributed to the different sources as follows:
#    https://notesfrompoland.com/2024/01/03/poland-produced-record-26-of-electricity-from-renewables-in-2023/
#    However, coal continued to account for almost two thirds of power production in Poland, with hard coal generating
#    65.27 TWh (43%) and brown coal (lignite) 31.47 TWh (20.8%). Gas accounted for a further 12.94 TWh (8.5%)
#    and oil 2.54 TWh (1.7%).

df_PL_hist['total'] = df_PL_hist['JWCD total generation'] + df_PL_hist['nJWCD total generation']

df_PL_hist['wind'] = df_PL_hist['Generation of Wind Farms']
df_PL_hist['pv'] = df_PL_hist['Generation of Solar Farms']

df_PL_hist['water'] = round(df_PL_hist['total'] * 0.013, 3)

df_PL_hist['thermal'] = df_PL_hist['total'] - df_PL_hist['wind'] - df_PL_hist['pv'] - df_PL_hist['water']

df_PL_hist['oil'] = round((2.54) / (65.27 + 31.47 + 12.94 + 2.54) * df_PL_hist['thermal'], 3)

df_PL_hist['gas'] = round((12.94) / (65.27 + 31.47 + 12.94 + 2.54) * df_PL_hist['thermal'], 3)

df_PL_hist['coal'] = round((65.27 + 31.47) / (65.27 + 31.47 + 12.94 + 2.54) * df_PL_hist['thermal'], 3)

df_PL_hist['biomass'] = 0

df_PL_hist['other'] = 0

df_PL_hist[['timestamp', 'total', 'wind', 'oil', 'gas', 'coal', 'biomass', 'pv', 'water']]

,timestamp,total,wind,oil,gas,coal,biomass,pv,water
0,2023-11-03 01:00:00,16455.713,5502.350,243.078,1238.356,9258.005,0,0.000,213.924
1,2023-11-03 02:00:00,16357.613,5502.100,240.892,1227.220,9174.752,0,0.000,212.649
2,2023-11-03 03:00:00,16430.863,5694.113,238.182,1213.416,9071.551,0,0.000,213.601
3,2023-11-03 04:00:00,16386.063,5481.388,241.996,1232.846,9216.813,0,0.000,213.019
4,2023-11-03 05:00:00,16378.876,5244.550,247.196,1259.338,9414.867,0,0.000,212.925
...,...,...,...,...,...,...,...,...,...
2779,2024-05-04 20:00:00,17359.413,389.975,360.687,1837.516,13737.350,0,808.213,225.672
2780,2024-05-04 21:00:00,17458.276,582.700,375.401,1912.475,14297.743,0,63.000,226.958
2781,2024-05-04 22:00:00,17029.013,833.988,361.549,1841.909,13770.190,0,0.000,221.377
2782,2024-05-04 23:00:00,14768.376,1004.088,307.197,1565.011,11700.091,0,0.000,191.989


In [ ]:
df_PL_hist[['timestamp', 'total', 'wind', 'oil', 'gas', 'coal', 'biomass', 'pv', 'water']].\
  to_csv(google_drive_path + "EnergyMix_PL_historical_estimated.csv", index = False)

## 2.Carbon emissions

In [ ]:
# Emission factors from electricitymaps.com
# https://www.electricitymaps.com/methodology?utm_source=app.electricitymaps.com&utm_medium=referral#carbon-intensity-and-emission-factors
# https://docs.google.com/spreadsheets/d/1ukTAD_oQKZfq-FgLpbLo_bGOv-UPTaoM_WS316xlDcE/edit?gid=316937240#gid=316937240

In [ ]:
emission_factors_df = pd.read_csv(google_drive_path + "ElectricityMapsEmissionFactors.csv")
emission_factors_df.head()

,zone_key,emission_factor_type,production_mode,datetime,value,source
0,CA-ON,lifecycle,gas,6/10/2024 0:00:00,497.0,"Mallia, E., Lewis, G. ""Life cycle greenhouse g..."
1,US-NW-BPAT,direct,gas,6/10/2024 0:00:00,385.5,eGrid 2021
2,AD,direct,gas,6/10/2024 0:00:00,370.0,IPCC 2014
3,AE,direct,gas,6/10/2024 0:00:00,370.0,IPCC 2014
4,AF,direct,gas,6/10/2024 0:00:00,370.0,IPCC 2014


In [ ]:
def get_carbon_footprint(energy_mix_df, emissions_df, country_code, emission_type):

  df = emissions_df[(emissions_df["zone_key"] == country_code) & (emissions_df['emission_factor_type'] == emission_type)]

  energy_mix_df["gCO2eq_footprint_per_KWh"] = energy_mix_df.apply(
      lambda row: (row.wind * df[df.production_mode == "wind"].iloc[0].value +
                  row.oil * df[df.production_mode == "oil"].iloc[0].value +
                  row.gas * df[df.production_mode == "gas"].iloc[0].value +
                  row.coal * df[df.production_mode == "coal"].iloc[0].value +
                  row.biomass * df[df.production_mode == "biomass"].iloc[0].value +
                  row.pv * df[df.production_mode == "solar"].iloc[0].value +
                  row.water * df[df.production_mode == "hydro discharge"].iloc[0].value) /
                  row.total,
      axis = 1
  )

  return energy_mix_df

### 2.1. Cyprus

In [ ]:
# Lifecycle emissions
get_carbon_footprint(get_cy_energy_mix_data(2024, 6, 17, 2024, 6, 17), emission_factors_df, "CY", "lifecycle").tail(5)

,timestamp,total,wind,oil,gas,coal,biomass,pv,water,gCO2eq_footprint_per_KWh
91,2024-06-17 22:45:00,804,3,791,0,0,10,0,0,941.098719
92,2024-06-17 23:00:00,794,3,781,0,0,10,0,0,940.941146
93,2024-06-17 23:15:00,781,3,768,0,0,10,0,0,940.730269
94,2024-06-17 23:30:00,769,4,755,0,0,10,0,0,939.305631
95,2024-06-17 23:45:00,757,5,741,0,0,10,0,0,936.576103


In [ ]:
get_carbon_footprint(pd.read_csv(google_drive_path + "EnergyMix_CY_historical_estimated.csv"), emission_factors_df, "CY", "lifecycle")

,timestamp,total,wind,oil,gas,coal,biomass,pv,water,gCO2eq_footprint_per_KWh
0,2023-11-03 00:00:00,445,8,427,0,0,10,0,0,920.432427
1,2023-11-03 00:15:00,437,9,418,0,0,10,0,0,917.671762
2,2023-11-03 00:30:00,431,10,411,0,0,10,0,0,914.988190
3,2023-11-03 00:45:00,425,9,406,0,0,10,0,0,916.657035
4,2023-11-03 01:00:00,419,6,403,0,0,10,0,0,922.865274
...,...,...,...,...,...,...,...,...,...,...
11131,2024-05-04 22:45:00,487,34,443,0,0,10,0,0,873.056078
11132,2024-05-04 23:00:00,474,42,422,0,0,10,0,0,854.965105
11133,2024-05-04 23:15:00,462,51,401,0,0,10,0,0,834.071926
11134,2024-05-04 23:30:00,449,36,403,0,0,10,0,0,862.047105


### 2.2. Poland

In [ ]:
get_carbon_footprint(get_pol_live_energy_mix_data(), emission_factors_df, "PL", "lifecycle")

,timestamp,total,wind,oil,gas,coal,biomass,pv,water,gCO2eq_footprint_per_KWh
0,1718987116138,18384,964,333,1694,12666,0,2443,284,918.463876


In [ ]:
get_carbon_footprint(pd.read_csv(google_drive_path + "EnergyMix_PL_historical_estimated.csv"), emission_factors_df, "PL", "lifecycle")

,timestamp,total,wind,oil,gas,coal,biomass,pv,water,gCO2eq_footprint_per_KWh
0,2023-11-03 01:00:00,16455.713,5502.350,243.078,1238.356,9258.005,0,0.000,213.924,750.215351
1,2023-11-03 02:00:00,16357.613,5502.100,240.892,1227.220,9174.752,0,0.000,212.649,747.999770
2,2023-11-03 03:00:00,16430.863,5694.113,238.182,1213.416,9071.551,0,0.000,213.601,736.658609
3,2023-11-03 04:00:00,16386.063,5481.388,241.996,1232.846,9216.813,0,0.000,213.019,750.057165
4,2023-11-03 05:00:00,16378.876,5244.550,247.196,1259.338,9414.867,0,0.000,212.925,765.992708
...,...,...,...,...,...,...,...,...,...,...
2779,2024-05-04 20:00:00,17359.413,389.975,360.687,1837.516,13737.350,0,808.213,225.672,1046.676201
2780,2024-05-04 21:00:00,17458.276,582.700,375.401,1912.475,14297.743,0,63.000,226.958,1081.390122
2781,2024-05-04 22:00:00,17029.013,833.988,361.549,1841.909,13770.190,0,0.000,221.377,1067.961792
2782,2024-05-04 23:00:00,14768.376,1004.088,307.197,1565.011,11700.091,0,0.000,191.989,1046.792124
